# 大量のモデルでのバッチ推論を並列で行うサンプル

テナントごとにモデルを作る必要があり、かつその数が大量である、というユースケースがあります。このようなユースケースにおいて、全てのモデルを使ってまとめてバッチ推論する際に、バッチ推論を並列化することで時間を短縮したいことがあります。このサンプルでは、SageMaker Processing Job を使って大量のモデルを使った並列バッチ推論を実現する方法を紹介します。

このサンプルでは、大量にあるモデルを3グループに分けて並列バッチ推論します。以下のようにモデルと入力ファイルをフォルダ分けしています。自身の環境で利用する場合は、入力ファイル名から対応するモデルを取得できるよう命名規則を決めておいてください。

```console
models
├── models1/
 | ├── model1.txt
 | └── model2.txt
├── models2/
└── models3/
```

```console
data
├── data1/
 | ├── data1.txt   // models1/model1 の入力ファイル
 | └── data2.txt   // models1/model2 の入力ファイル
├── data2/        // models2 の中のモデルの入力ファイル
└── data3/        // models3 の中のモデルの入力ファイル
```


## Prepare resources



In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
# from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import Processor
from sagemaker.image_uris import retrieve
from datetime import datetime
from dateutil import tz
import os
import time

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = get_execution_role()
JST = tz.gettz('Asia/Tokyo')

project_name = 'sagemaker-processing-parallel'
user_name = 'demo'

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/proctest'

Processing Job の設定を作っていきます。`Processor` クラスに、使用するコンテナイメージのURL、コンテナ実行時に実行するコマンド、使用するインスタンスタイプと数などを設定します。

In [ ]:

code_path = '/opt/ml/processing/input/code'
input_path = '/opt/ml/processing/input/data'
model_path = '/opt/ml/processing/input/model'
output_path = '/opt/ml/processing/output/data'

image_uri = retrieve(
    framework='sklearn',
    version='1.0-1',
    region=region,
    py_version='py3',
    instance_type='ml.m5.xlarge'
)

processor = Processor(
    image_uri=image_uri,
    entrypoint=["python3", f"{code_path}/predict.py"],
    role=role,
    instance_count=1,
        instance_type="ml.m5.xlarge"
)

## Upload data
モデルファイルを想定した models 以下のファイルを S3 にアップロード

In [ ]:
data_dir = 'models'
models = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix+'/models')
print('input spec (in this case, just an S3 path): {}'.format(inputs))

推論データを想定した data 以下のファイルを S3 にアップロード

In [ ]:
data_dir = 'data'
inputs = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix+'/data')
print('input spec (in this case, just an S3 path): {}'.format(inputs))

Processing Job 内で使用するコードを S3 にアップロード

In [ ]:
data_dir = 'code'
code_s3_path = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix+'/code')
print('input spec (in this case, just an S3 path): {}'.format(inputs))

## Run Processing job

このサンプルはモデルファイルが3つのパスに分割して保存されている想定のため Processing Job の起動を 3回ループして実行します。実行時に、モデルファイルと入力ファイルのパスを変更します。以下のセルを実行すると、3つの SageMaker Processing Job が起動します。Job の実行は 4分程度で終了します。

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

timestamp = datetime.now(JST).strftime('%Y%m%d-%H%M%S')

model_dirs = [('models1','data1'), ('models2','data2'), ('models3','data3')]

for i, (model_dir, data_dir) in enumerate(model_dirs):
    print('Start inference job')
    s3_model_path = os.path.join(models, model_dir)
    print(s3_model_path)
    s3_data_path = os.path.join(inputs, data_dir)
    print(s3_data_path)
    
    job_base_name =  project_name + '-' + user_name + '-' + timestamp
    job_name = job_base_name + str(i)
    output_s3_path = f's3://{bucket}/{prefix}/{job_base_name}/output/result'

    processor.run(
        job_name=job_name,
        inputs=[
            ProcessingInput(
                input_name='code',
                source=code_s3_path,
                destination=code_path),
            ProcessingInput(
                input_name='data',
                source=s3_data_path,
                destination=input_path),
        ProcessingInput(
                input_name='models',
                source=s3_model_path,
                destination=model_path)],
        outputs=[
            ProcessingOutput(source=output_path, destination=output_s3_path),
        ],
         arguments=['--code-path', code_path,
                  '--input-data-path', input_path,
                     '--model-path', model_path,
                     '--job-id', str(i),
                  '--output-data-path', output_path],
        logs=False,
        wait=False
    )
    time.sleep(3)